# COMPAS 数据统计分析项目

本 Notebook 基于 COMPAS 数据集（compas-scores-raw.csv），主要完成以下任务：

1. 按 `Ethnic_Code_Text` 分组，计算 `DecileScore` 的基本统计信息，并绘制条形图展示均值及 95% 置信区间。
2. 按 `Sex_Code_Text` 及（如果存在）`RecSupervisionLevelText` 分组，计算 `DecileScore` 的统计数据，并绘制图表。
3. 探讨样本量对 95% 置信区间的影响，理解样本量与统计推断精度之间的关系。

下面每一部分代码均带有详细注释和反思，解释各部分代码的用途、关键功能以及它们与统计原理的关系。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 如果之前为显示中文设置过字体，现在要让图表以英文为主，则保持以下两行注释状态
# plt.rcParams['font.sans-serif'] = ['SimHei']
# plt.rcParams['axes.unicode_minus'] = False

print("成功导入所需的库")

: 

## 1. 数据加载与初步探索

首先，我们指定数据集路径，加载 COMPAS 数据，并打印数据预览及列名以确认数据结构。

反思：了解数据的基本结构和字段名称对后续数据清洗和分析至关重要。

In [ ]:
# 数据集路径（请确保路径正确）
data_path = '/Users/xiangxiaoxin/Documents/GitHub/profile_intro_datascience/week6_classification_decisiontree/data/compas-scores-raw.csv'

# 加载数据
df = pd.read_csv(data_path)

# 打印数据预览
print("数据预览：")
print(df.head())

# 打印所有列名，帮助确认数据结构
print("\n列名：")
print(df.columns)

## 2. Task 1: 按 Ethnic_Code_Text 分组计算 DecileScore 的统计信息

本部分代码将数据按照 `Ethnic_Code_Text` 分组，并计算每组的均值、中位数、标准差、样本数量以及范围。最后还计算 95% 置信区间（CI），并以条形图展示各组的平均 DecileScore 与置信区间。

反思：统计汇总（如均值和置信区间）有助于我们了解不同种族组的评分分布情况，较窄的置信区间意味着对总体参数的估计较为精确。

In [ ]:
# 按 Ethnic_Code_Text 分组
grouped_ethnic = df.groupby('Ethnic_Code_Text')

# 计算每组的均值、中位数、标准差和样本数量
stats_ethnic = grouped_ethnic['DecileScore'].agg(['mean', 'median', 'std', 'count'])

# 计算范围：最大值减去最小值
stats_ethnic['range'] = grouped_ethnic['DecileScore'].max() - grouped_ethnic['DecileScore'].min()

# 计算 95% 置信区间，公式：CI = 1.96 * (std / sqrt(n))
z = 1.96
stats_ethnic['CI'] = z * (stats_ethnic['std'] / np.sqrt(stats_ethnic['count']))

print("\nDecileScore 统计数据（按 Ethnic_Code_Text 分组）：")
print(stats_ethnic)

# 绘制条形图：展示各种族组的平均 DecileScore 及其 95% 置信区间
plt.figure(figsize=(10, 6))
plt.bar(stats_ethnic.index, stats_ethnic['mean'], yerr=stats_ethnic['CI'], capsize=5, color='skyblue')
plt.xlabel('Ethnicity (Ethnic_Code_Text)')
plt.ylabel('Mean DecileScore')
plt.title('Mean DecileScore by Ethnicity (with 95% CI)')
plt.show()

### 反思：统计汇总与置信区间的意义

通过上述汇总，我们可以看到不同种族组的 DecileScore 均值及其波动情况。置信区间反映了样本均值对总体均值估计的精度，样本量越大，置信区间越窄，表明估计更稳定。

## 3. Task 2: 按 Sex_Code_Text 和 RecSupervisionLevelText 分组统计

本部分代码根据 `Sex_Code_Text` 及（如果存在）`RecSupervisionLevelText` 对数据进行分组，计算 `DecileScore` 的均值、标准差和样本数量，并计算 95% 置信区间。随后，通过 unstack() 将结果整理为适合绘图的格式，并生成条形图。

反思：分组统计有助于我们比较不同子群体（如不同性别和监督级别）的评分差异，从而揭示数据中潜在的偏差或差异。

In [ ]:
# 判断是否存在 RecSupervisionLevelText 字段，若存在则两字段分组，否则仅按 Sex_Code_Text 分组
if 'RecSupervisionLevelText' in df.columns:
    group_cols = ['Sex_Code_Text', 'RecSupervisionLevelText']
else:
    group_cols = ['Sex_Code_Text']

grouped_cross = df.groupby(group_cols)
stats_cross = grouped_cross['DecileScore'].agg(['mean', 'std', 'count'])
stats_cross['CI'] = z * (stats_cross['std'] / np.sqrt(stats_cross['count']))

print("\nDecileScore 统计数据（按 Sex_Code_Text 和 RecSupervisionLevelText 分组）：")
print(stats_cross)

# 根据分组列的数量，选择不同的绘图方式
if len(group_cols) > 1:
    pivot_mean = stats_cross['mean'].unstack(group_cols[1])
    pivot_CI = stats_cross['CI'].unstack(group_cols[1])
    
    pivot_mean.plot(kind='bar', yerr=pivot_CI, capsize=4, figsize=(10, 6))
    plt.xlabel('Sex (Sex_Code_Text)')
    plt.ylabel('Mean DecileScore')
    plt.title('Mean DecileScore by Sex and Supervision Level (with 95% CI)')
    plt.legend(title=group_cols[1])
    plt.show()
else:
    plt.figure(figsize=(10, 6))
    plt.bar(stats_cross.index, stats_cross['mean'], yerr=stats_cross['CI'], capsize=5, color='lightgreen')
    plt.xlabel('Sex (Sex_Code_Text)')
    plt.ylabel('Mean DecileScore')
    plt.title('Mean DecileScore by Sex (with 95% CI)')
    plt.show()


## 4. Task 4: 探讨样本量对 95% 置信区间的影响

本部分中，我们定义了一个函数 `compute_CI`，用于计算给定样本中指定列的均值及 95% 置信区间。通过对不同样本量（例如 30 和 300）的抽样计算，可以观察到样本量越大，置信区间越窄，从而验证统计原理中标准误差与样本量成反比的关系。

反思：样本量越大，统计量的波动越小，估计越精确，这为我们在实际中如何选择合适的样本量提供了理论支持。

In [ ]:
def compute_CI(sample, col, z=1.96):
    """计算给定样本中指定列的均值及 95% 置信区间"""
    mean_val = sample[col].mean()
    std_val = sample[col].std()
    n = sample.shape[0]
    error = z * (std_val / np.sqrt(n))
    return mean_val, error

sample_sizes = [30, 300]

print("\n样本量对 DecileScore 95% CI 的影响：")
for n in sample_sizes:
    sample = df.sample(n, random_state=42)
    mean_val, error = compute_CI(sample, 'DecileScore')
    print(f"样本量: {n}")
    print(f"DecileScore 均值: {mean_val:.2f}")
    print(f"95% CI: {mean_val:.2f} ± {error:.2f} (区间宽度: {error*2:.2f})")
    print("-" * 40)


## 5. 反思与总结

在本 Notebook 中，我们基于 COMPAS 数据集进行了统计分析：

- 通过按种族、性别（和监督级别）分组计算 `DecileScore` 的统计量，了解不同子群体之间的差异；
- 利用置信区间展示各组估计值的精度，验证样本量对统计推断的影响；
- 通过实验，我们验证了较大样本量会使置信区间变窄，说明统计量更稳定。

这些统计方法为我们从数据中提取见解提供了有力工具，帮助我们在实际应用中更好地了解数据分布和变化情况。